This exercise will require you to pull some data from the NASDAQ Data Link API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'sT_LmuA5BapzBSvRw2pB'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Hong Kong Stock Exhange (HKEX), which is available for free. We'll try and analyze the stock prices of a company called Meta Media Holdings Limited, formerly Modern Media Holdings Limited is an investment holding company mainly engaged in the media businesses.The Company operates its business through two segments. The Print Media and Art Platform segment engages in the sale of advertising space in the publication of and the distribution of the Company's magazines and periodicals, artwork trading and auction, art exhibition and education and revenue from restaurant operation. The Digital Platform segment operates digital media platform in which the Company publishes digital media products and sells advertising spaces, and engages in the production of customized contents for brand advertisers. : https://modernmedia.com.cn/. The company is listed under the stock ticker 72.

You can find the detailed NASDAQ Data Link API (formerly Quandl API) instructions here: https://docs.data.nasdaq.com/docs/time-series

While there is a dedicated Python package for connecting to the NASDAQ Data Link API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [73]:
# First, import the relevant modules
import requests
import collections
import json
import numpy as np
import statistics

Note: API's can change a bit with each version, for this exercise it is reccomended to use the "V3" quandl api at `https://www.quandl.com/api/v3/`

In [3]:
# Now, call the NASDAQ Date Link Time-Series API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = "https://data.nasdaq.com/api/v3/datasets/HKEX/00072/data.json?collapse=daily&rows=1&order=asc&column_index=1&api_key=sT_LmuA5BapzBSvRw2pB"
r = requests.get(url)
r.text

'{"dataset_data":{"limit":1,"transform":null,"column_index":1,"column_names":["Date","Nominal Price"],"start_date":"2014-02-21","end_date":"2023-06-30","frequency":"daily","data":[["2014-02-21",1.44]],"collapse":"daily","order":"asc"}}'

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.status_code  ##returned valide code of 200
r.headers['content-type'] ##returned 'application/json; charset=utf-8'
r.encoding ## returned 'utf-8'
r.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': 1,
  'column_names': ['Date', 'Nominal Price'],
  'start_date': '2014-02-21',
  'end_date': '2023-06-30',
  'frequency': 'daily',
  'data': [['2014-02-21', 1.44]],
  'collapse': 'daily',
  'order': 'asc'}}

These are your tasks for this mini project:

1. Collect data from the Hong Kong Stock Exchange, for the ticker 72, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
## TASK 1: Collect data from the Hong Kong Stock Exchange, for the ticker 72, for the whole year 2017.
url_2017 = "https://data.nasdaq.com/api/v3/datasets/HKEX/00072/data.json?collapse=daily&start_date='2017-01-01'&end_date='2017-12-31'&order=asc&api_key=sT_LmuA5BapzBSvRw2pB"
r_2017 = requests.get(url_2017)
r_2017.text
# r_2017 result below shows a dictionary of lists with each list being one trading day during the Calendar Year 2017.
# The data columns are Date, Nominal Price, Net Change, Change (%), Bid, Ask, P/E(x), 
# High, Low, Previous Close, Share Volume (000), Turnover (000), and Lot Size. 

'{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Nominal Price","Net Change","Change (%)","Bid","Ask","P/E(x)","High","Low","Previous Close","Share Volume (000)","Turnover (000)","Lot Size"],"start_date":"2017-01-01","end_date":"2017-12-31","frequency":"daily","data":[["2017-01-03",1.02,0.04,4.082,0.95,1.02,17.86,1.02,0.97,0.98,40.0,40.0,2000.0],["2017-01-04",1.0,0.02,1.961,0.95,1.0,17.51,null,null,1.02,0.0,0.0,2000.0],["2017-01-05",1.0,0.0,0.0,0.94,1.01,17.51,null,null,1.0,0.0,0.0,2000.0],["2017-01-06",1.04,0.04,4.0,0.97,1.04,18.21,1.04,1.0,1.0,22.0,22.0,2000.0],["2017-01-09",1.06,0.02,1.923,0.95,1.06,18.56,1.06,1.02,1.04,30.0,31.0,2000.0],["2017-01-10",1.07,0.01,0.943,0.95,1.09,18.74,1.07,1.06,1.06,12.0,13.0,2000.0],["2017-01-11",1.08,0.01,0.935,0.98,1.08,18.91,1.08,1.06,1.07,24.0,26.0,2000.0],["2017-01-12",1.08,0.0,0.0,0.96,1.08,18.91,null,null,1.08,0.0,0.0,2000.0],["2017-01-13",1.06,0.02,1.852,0.97,1.06,18.56,null,null,1.08,0.0,0.0,2000.0]

In [6]:
#TASK 2: Convert the returned JSON object into a Python dictionary.
MDH_data = r_2017.json()
type(MDH_data)

dict

In [7]:
## TASK 3: Calculate what the highest and lowest opening prices were for the stock in this period.

In [8]:
MDH_data.keys()

dict_keys(['dataset_data'])

In [9]:
MDH_data['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [10]:
MDH_data['dataset_data']['column_names']

['Date',
 'Nominal Price',
 'Net Change',
 'Change (%)',
 'Bid',
 'Ask',
 'P/E(x)',
 'High',
 'Low',
 'Previous Close',
 'Share Volume (000)',
 'Turnover (000)',
 'Lot Size']

In [11]:
MDH_data['dataset_data']['column_names'][1]

'Nominal Price'

In [12]:
MDH_data['dataset_data']['data']

[['2017-01-03',
  1.02,
  0.04,
  4.082,
  0.95,
  1.02,
  17.86,
  1.02,
  0.97,
  0.98,
  40.0,
  40.0,
  2000.0],
 ['2017-01-04',
  1.0,
  0.02,
  1.961,
  0.95,
  1.0,
  17.51,
  None,
  None,
  1.02,
  0.0,
  0.0,
  2000.0],
 ['2017-01-05',
  1.0,
  0.0,
  0.0,
  0.94,
  1.01,
  17.51,
  None,
  None,
  1.0,
  0.0,
  0.0,
  2000.0],
 ['2017-01-06',
  1.04,
  0.04,
  4.0,
  0.97,
  1.04,
  18.21,
  1.04,
  1.0,
  1.0,
  22.0,
  22.0,
  2000.0],
 ['2017-01-09',
  1.06,
  0.02,
  1.923,
  0.95,
  1.06,
  18.56,
  1.06,
  1.02,
  1.04,
  30.0,
  31.0,
  2000.0],
 ['2017-01-10',
  1.07,
  0.01,
  0.943,
  0.95,
  1.09,
  18.74,
  1.07,
  1.06,
  1.06,
  12.0,
  13.0,
  2000.0],
 ['2017-01-11',
  1.08,
  0.01,
  0.935,
  0.98,
  1.08,
  18.91,
  1.08,
  1.06,
  1.07,
  24.0,
  26.0,
  2000.0],
 ['2017-01-12',
  1.08,
  0.0,
  0.0,
  0.96,
  1.08,
  18.91,
  None,
  None,
  1.08,
  0.0,
  0.0,
  2000.0],
 ['2017-01-13',
  1.06,
  0.02,
  1.852,
  0.97,
  1.06,
  18.56,
  None,
  None,
  

In [13]:
#Nominal Price is the Opening Price
open_price = MDH_data['dataset_data']['data'][5][1]
print(open_price)
type(open_price)

1.07


float

In [14]:
Opening_Price = MDH_data['dataset_data']['column_names'][1], MDH_data['dataset_data']['data'][5][:2]
print (Opening_Price)

('Nominal Price', ['2017-01-10', 1.07])


In [15]:
type(Opening_Price)

tuple

In [16]:
Opening_Price_List = []
for day in MDH_data['dataset_data']['data']:
    price = day[1]
    Opening_Price_List.append(price)
print(Opening_Price_List)
len(Opening_Price_List)
type(price)
    

[1.02, 1.0, 1.0, 1.04, 1.06, 1.07, 1.08, 1.08, 1.06, 1.04, 1.01, 1.01, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.15, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.11, 1.1, 1.1, 1.1, 1.09, 1.09, 1.08, 1.08, 1.1, 1.11, 1.11, 1.11, 1.1, 1.1, 1.1, 1.1, 1.11, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.05, 0.98, 1.09, 1.09, 1.09, 1.09, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.12, 1.1, 1.1, 1.1, 1.1, 1.1, 1.09, 1.09, 1.09, 1.01, 0.96, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.09, 1.09, 1.09, 1.09, 1.09, 1.07, 1.07, 1.08, 1.08, 1.07, 1.07, 1.09, 1.09, 1.09, 1.04, 1.04, 1.04, 1.04, 1.04, 1.04, 1.04, 1.04, 1.05, 1.04, 1.04, 1.04, 1.04, 1.03, 1.03, 1.03, 1.03, 1.02, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.94, 0.99, 0.99, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.0, 1.0, 1.0, 1.0, 0.98, 1.02, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.94, 0.89, 0.95

float

In [17]:
highest_opening_price = max(Opening_Price_List)
lowest_opening_price = min(Opening_Price_List)
print("72 High Open in 2017 of 245 days recorded:", highest_opening_price)
print("72 Low Open in 2017 of 245 days recorded:", lowest_opening_price)

72 High Open in 2017 of 245 days recorded: 1.15
72 Low Open in 2017 of 245 days recorded: 0.6


In [18]:
## TASK 4: What was the largest change in any one day (based on High and Low price)?

In [52]:
High_List = []
for day in MDH_data['dataset_data']['data']:
    high = day[7]
    if high != None:
        High_List.append(high)
    else:
        High_List.append(0)
print(High_List)
len(High_List)

[1.02, 0, 0, 1.04, 1.06, 1.07, 1.08, 0, 0, 1.04, 0, 1.01, 1.15, 0, 0, 0, 0, 0, 0, 1.1, 0, 0, 0, 1.1, 1.1, 1.1, 0, 0, 1.12, 1.11, 0, 0, 1.1, 1.1, 0, 1.08, 1.1, 1.11, 0, 0, 1.1, 0, 0, 0, 1.11, 1.1, 0, 0, 0, 1.1, 0, 1.1, 0, 0, 1.11, 1.04, 1.01, 1.09, 1.09, 1.09, 0, 1.12, 0, 1.12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.02, 0.96, 1.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.1, 1.1, 0, 0, 0, 0, 1.08, 0, 1.09, 0, 1.08, 1.07, 1.1, 0, 0, 0, 1.04, 0, 0, 0, 0, 0, 0, 1.12, 1.1, 0, 0, 0, 0, 0, 0, 0, 1.03, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0.93, 1.0, 0, 1.04, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0.98, 1.04, 1.0, 1.01, 0, 0, 0, 0, 0.95, 0.96, 0.96, 0, 0.95, 0.94, 0.94, 0.87, 0.86, 0.89, 0, 0.85, 0, 0, 0.88, 0.88, 0.87, 0, 0.89, 0.83, 0, 0, 0.82, 0.91, 0.82, 0.85, 0, 0, 0.83, 0.86, 0, 0, 0.8, 0.79, 0.78, 0.75, 0.78, 0, 0, 0, 0.76, 0.72, 0.72, 0.75, 0, 0.75, 0.78, 0.7, 0.71, 0.64, 0.67, 0.66, 0.65, 0, 0.75, 0.7, 0.67, 0, 0, 0, 0, 0.69, 0.7, 0, 0.67, 0

245

In [53]:
type(MDH_data['dataset_data']['data'][1][7])

NoneType

In [54]:
Low_List = []
for day in MDH_data['dataset_data']['data']:
    low = day[8]
    if low != None:
        Low_List.append(low)
    else:
        Low_List.append(0)
print(Low_List)
len(Low_List)

[0.97, 0, 0, 1.0, 1.02, 1.06, 1.06, 0, 0, 0.94, 0, 1.01, 1.0, 0, 0, 0, 0, 0, 0, 1.1, 0, 0, 0, 1.1, 1.1, 1.1, 0, 0, 1.04, 1.01, 0, 0, 1.05, 1.09, 0, 1.05, 1.1, 1.1, 0, 0, 1.1, 0, 0, 0, 1.11, 1.1, 0, 0, 0, 1.1, 0, 1.1, 0, 0, 1.1, 1.04, 0.89, 0.98, 1.0, 1.0, 0, 1.12, 0, 1.12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.99, 0.96, 0.96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.1, 1.1, 0, 0, 0, 0, 0.98, 0, 1.09, 0, 0.97, 0.98, 1.1, 0, 0, 0, 1.04, 0, 0, 0, 0, 0, 0, 0.98, 1.04, 0, 0, 0, 0, 0, 0, 0, 1.01, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0.93, 0.99, 0, 1.04, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0.9, 0.89, 0.91, 1.01, 0, 0, 0, 0, 0.93, 0.85, 0.86, 0, 0.9, 0.87, 0.88, 0.87, 0.85, 0.78, 0, 0.85, 0, 0, 0.82, 0.88, 0.87, 0, 0.89, 0.81, 0, 0, 0.79, 0.81, 0.82, 0.85, 0, 0, 0.83, 0.86, 0, 0, 0.75, 0.73, 0.78, 0.75, 0.78, 0, 0, 0, 0.76, 0.72, 0.7, 0.7, 0, 0.7, 0.7, 0.66, 0.6, 0.61, 0.64, 0.66, 0.65, 0, 0.67, 0.68, 0.67, 0, 0, 0, 0, 0.69, 0.7, 0, 0.6, 0.65, 

245

In [55]:
np_high = np.array(High_List)
np_low = np.array(Low_List)
daily_change = np_high - np_low

In [58]:
print(daily_change)

[0.05 0.   0.   0.04 0.04 0.01 0.02 0.   0.   0.1  0.   0.   0.15 0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.08 0.1  0.   0.   0.05 0.01 0.   0.03 0.   0.01 0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.
 0.12 0.11 0.09 0.09 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.03 0.   0.14 0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.1  0.   0.
 0.   0.11 0.09 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.14 0.06 0.   0.   0.   0.   0.   0.   0.   0.02 0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.08 0.15 0.09 0.   0.   0.   0.   0.   0.02 0.11 0.1  0.   0.05 0.07
 0.06 0.   0.01 0.11 0.   0.   0.   0.   0.06 0.   0.   0.   0.   0.02
 0.   0.   0.03 0.1  0.   0.

In [61]:
print("Largest change in any one day (based on High and Low price):", max(daily_change))

Largest change in any one day (based on High and Low price): 0.15000000000000002


In [62]:
## Task 5: What was the largest change between any two days (based on Closing Price)?

In [66]:
Net_Change_List = []
for day in MDH_data['dataset_data']['data']:
    change = day[2]
    if change != None:
        Net_Change_List.append(change)
    else:
        Net_Change_List.append(0)
print(Net_Change_List)
len(Net_Change_List)    

[0.04, 0.02, 0.0, 0.04, 0.02, 0.01, 0.01, 0.0, 0.02, 0.02, 0.03, 0.0, 0.14, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.0, 0.0, 0.01, 0.0, 0.01, 0.0, 0.02, 0.01, 0.0, 0.0, 0.01, 0.0, 0.0, 0.0, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.07, 0.11, 0.0, 0.0, 0.0, 0.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.08, -0.05, 0.14, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.01, 0.0, 0.0, 0.0, 0.0, -0.02, 0.0, 0.01, 0.0, -0.01, 0.0, 0.02, 0.0, 0.0, -0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, -0.01, 0.0, 0.0, 0.0, -0.01, 0.0, 0.0, 0.0, -0.01, -0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.06, 0.05, 0.0, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.03, 0.0, 0.0, 0.0, -0.02, 0.04, -0.02, 0.0, 0.0, 0.0, 0.0, 0.0, -0.06, -0.05, 0.06, 0.0, 0.0, -0.04, 0.0, -0.03, -0.03, 0.

245

In [68]:
print("The Largest change between any two days (based on Closing Price):", max(Net_Change_List))

The Largest change between any two days (based on Closing Price): 0.14


In [69]:
## Task 6: What was the average daily trading volume during this year?
# In this dataset the column "Turnover (000)" contains these values by day.

In [71]:
Daily_Trading_Volume_List = []
for day in MDH_data['dataset_data']['data']:
    volume = day[11]
    if volume != None:
        Daily_Trading_Volume_List.append(volume)
    else:
        Daily_Trading_Volume_List.append(0)
print(Daily_Trading_Volume_List)
len(Daily_Trading_Volume_List) 

[40.0, 0.0, 0.0, 22.0, 31.0, 13.0, 26.0, 0.0, 0.0, 61.0, 0.0, 40.0, 225.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 33.0, 0.0, 0.0, 0.0, 18.0, 7.0, 20.0, 0.0, 0.0, 26.0, 41.0, 0.0, 0.0, 6.0, 33.0, 0.0, 4.0, 4.0, 7.0, 0.0, 0.0, 11.0, 0.0, 0.0, 0.0, 13.0, 33.0, 0.0, 0.0, 0.0, 11.0, 1.0, 9.0, 0.0, 0.0, 62.0, 10.0, 578.0, 207.0, 15.0, 65.0, 0.0, 11.0, 0.0, 189.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 120.0, 19.0, 85.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 23.0, 0.0, 4.0, 0.0, 228.0, 125.0, 4.0, 58.0, 0.0, 0.0, 314.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 234.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1165.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 88.0, 33.0, 438.0, 20.0, 0.0, 0.0, 0.0, 0.0, 40.0, 544.0, 198.0, 0.0, 65.0, 334.0, 77.0, 21.0, 62.0, 60.0, 0

245

In [75]:
print("The average daily trading volume during 2017:", statistics.mean(Daily_Trading_Volume_List))

The average daily trading volume during 2017: 41.481632653061226


In [77]:
## Task 7: (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [80]:
Daily_Trading_Volume_List2 = []
for day in MDH_data['dataset_data']['data']:
    volume = day[11]
    if volume != None:
        Daily_Trading_Volume_List2.append(volume)
    else:
        Daily_Trading_Volume_List2 = Daily_Trading_Volume_List2
print(Daily_Trading_Volume_List2)
len(Daily_Trading_Volume_List2) 

[40.0, 0.0, 0.0, 22.0, 31.0, 13.0, 26.0, 0.0, 0.0, 61.0, 0.0, 40.0, 225.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 33.0, 0.0, 0.0, 0.0, 18.0, 7.0, 20.0, 0.0, 0.0, 26.0, 41.0, 0.0, 0.0, 6.0, 33.0, 0.0, 4.0, 4.0, 7.0, 0.0, 0.0, 11.0, 0.0, 0.0, 0.0, 13.0, 33.0, 0.0, 0.0, 0.0, 11.0, 1.0, 9.0, 0.0, 0.0, 62.0, 10.0, 578.0, 207.0, 15.0, 65.0, 0.0, 11.0, 0.0, 189.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 120.0, 19.0, 85.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 23.0, 0.0, 4.0, 0.0, 228.0, 125.0, 4.0, 58.0, 0.0, 0.0, 314.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 234.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1165.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 14.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 88.0, 33.0, 438.0, 20.0, 0.0, 0.0, 0.0, 0.0, 40.0, 544.0, 198.0, 0.0, 65.0, 334.0, 77.0, 21.0, 62.0, 60.0, 0

245

In [78]:
print("The median trading volume during 2017:", statistics.median(Daily_Trading_Volume_List))

The median trading volume during 2017: 0.0


In [81]:
##Not a very active stock in terms of trading volume throughout 2017.
##In this case values were entered for each day (i.e. no occurances of None in the data)